In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

In [2]:
RANDOM_SEED = 99

In [ ]:
tmp = pd.read_parquet("./data/test_v2_2.parquet")
tmp["V_WIND"].isna().sum()

In [3]:
train_df = pd.read_parquet('./data/train_v2_2_nn.parquet').drop(columns=["U_WIND"])
train_null_df = pd.read_parquet('./data/train_v2_2_n.parquet').drop(columns=["U_WIND", "V_WIND"])
test_df = pd.read_parquet('./data/test_v2_2.parquet').drop(columns=["U_WIND", "V_WIND"])

In [4]:
train_x = train_df.drop(columns=["V_WIND"])
train_y = train_df["V_WIND"]

In [5]:
def create_model(param):
    return xgb.XGBRegressor(
        random_state=RANDOM_SEED,
        tree_method="gpu_hist",
        eval_metric="rmse",
        early_stopping_rounds=100,
        **param
    )

In [6]:
def train_model(param, X, y, X_test, trial=None, test=False):
  skf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

  val_scores = []
  y_tests = []
  models = []

  for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model = create_model(param)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

    y_hat_val = model.predict(X_val)
    score = mean_squared_error(y_val.values, y_hat_val, squared=False)
    val_scores.append(score)
    print(f'Fold: {idx+1}/3 rmse score = {score:.5f}')

    if test:
      y_tests.append(model.predict(X_test))
      models.append(model)

    if trial:
      trial.report(score, idx)

      if trial.should_prune():
        raise optuna.TrialPruned()

  return val_scores, y_tests, models

In [7]:
def objective_xgb(trial):
  param = {
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
    "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
  }

  if param["booster"] == "dart":
    param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
    param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
    param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
    param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

  val_scores, y_tests, models = train_model(param, train_x, train_y, test_df, trial)

  return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
      min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=50)

[I 2023-10-10 02:34:29,905] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:3.81383	validation_1-rmse:3.80904
[299]	validation_0-rmse:3.81383	validation_1-rmse:3.80904
Fold: 1/3 rmse score = 3.80904
[0]	validation_0-rmse:3.80515	validation_1-rmse:3.82636
[299]	validation_0-rmse:3.80515	validation_1-rmse:3.82636
Fold: 2/3 rmse score = 3.82636
[0]	validation_0-rmse:3.81771	validation_1-rmse:3.80126
[299]	validation_0-rmse:3.81771	validation_1-rmse:3.80126


[I 2023-10-10 02:34:41,698] Trial 0 finished with value: 3.8122195634844673 and parameters: {'booster': 'gbtree', 'lambda': 0.04017550942202233, 'alpha': 1.7847241527490656e-08, 'subsample': 0.8272449670283629, 'colsample_bytree': 0.6090556776494775, 'n_estimators': 300, 'max_depth': 3, 'min_child_weight': 10, 'eta': 1.1339812255777714e-08, 'gamma': 0.014399394343769443, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 3.8122195634844673.


Fold: 3/3 rmse score = 3.80126
[0]	validation_0-rmse:3.81383	validation_1-rmse:3.80904
[899]	validation_0-rmse:3.81306	validation_1-rmse:3.80827
Fold: 1/3 rmse score = 3.80827
[0]	validation_0-rmse:3.80515	validation_1-rmse:3.82636
[899]	validation_0-rmse:3.80438	validation_1-rmse:3.82559
Fold: 2/3 rmse score = 3.82559
[0]	validation_0-rmse:3.81771	validation_1-rmse:3.80126
[899]	validation_0-rmse:3.81693	validation_1-rmse:3.80049


[I 2023-10-10 03:07:43,411] Trial 1 finished with value: 3.8114476078947725 and parameters: {'booster': 'dart', 'lambda': 1.4575798463318516e-05, 'alpha': 0.6189425995687715, 'subsample': 0.5719732438377161, 'colsample_bytree': 0.1842517839679773, 'n_estimators': 900, 'max_depth': 4, 'min_child_weight': 6, 'eta': 2.178476742104051e-06, 'gamma': 0.033817296080697445, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.4193450623251104e-07, 'skip_drop': 4.983900949448835e-07}. Best is trial 1 with value: 3.8114476078947725.


...

Fold: 3/3 rmse score = 3.00077
[0]	validation_0-rmse:3.78090	validation_1-rmse:3.77639
[999]	validation_0-rmse:2.57319	validation_1-rmse:2.82366
Fold: 1/3 rmse score = 2.82366
[0]	validation_0-rmse:3.77245	validation_1-rmse:3.79331
[999]	validation_0-rmse:2.57376	validation_1-rmse:2.82628
Fold: 2/3 rmse score = 2.82628
[0]	validation_0-rmse:3.78423	validation_1-rmse:3.76870
[999]	validation_0-rmse:2.57290	validation_1-rmse:2.83525


[I 2023-10-10 10:10:39,886] Trial 48 finished with value: 2.8283956264886907 and parameters: {'booster': 'dart', 'lambda': 0.16289505989975228, 'alpha': 1.1189457100152212e-06, 'subsample': 0.73079940836495, 'colsample_bytree': 0.5077120814595937, 'n_estimators': 1000, 'max_depth': 7, 'min_child_weight': 2, 'eta': 0.029547059124562885, 'gamma': 0.11737986014049712, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 2.1624208980512185e-07, 'skip_drop': 0.06833916733278053}. Best is trial 41 with value: 2.7127541394945687.


Fold: 3/3 rmse score = 2.83525
[0]	validation_0-rmse:3.71582	validation_1-rmse:3.71039
[899]	validation_0-rmse:2.34907	validation_1-rmse:2.77223
Fold: 1/3 rmse score = 2.77219
[0]	validation_0-rmse:3.70696	validation_1-rmse:3.72717
[899]	validation_0-rmse:2.35618	validation_1-rmse:2.77645
Fold: 2/3 rmse score = 2.77645
[0]	validation_0-rmse:3.71759	validation_1-rmse:3.70388
[899]	validation_0-rmse:2.34904	validation_1-rmse:2.78659


[I 2023-10-10 10:43:05,946] Trial 49 finished with value: 2.778389977616566 and parameters: {'booster': 'dart', 'lambda': 0.005775964181075006, 'alpha': 5.851670779130156e-06, 'subsample': 0.7569654196045118, 'colsample_bytree': 0.5859999109027066, 'n_estimators': 900, 'max_depth': 6, 'min_child_weight': 4, 'eta': 0.09811762107771854, 'gamma': 0.8695862365231523, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 8.837789830769114e-08, 'skip_drop': 0.10515605394602845}. Best is trial 41 with value: 2.7127541394945687.


Fold: 3/3 rmse score = 2.78653


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'booster': 'dart', 'lambda': 0.16575861086319754, 'alpha': 4.7672202249640596e-07, 'subsample': 0.7872382459364554, 'colsample_bytree': 0.5654477668877006, 'n_estimators': 1000, 'max_depth': 7, 'min_child_weight': 4, 'eta': 0.0977204872349886, 'gamma': 0.22036177520409278, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.0763390413982438e-08, 'skip_drop': 0.7285832530365487}


In [11]:
best_param = {
  'booster': 'dart',
  'lambda': 0.16575861086319754,
  'alpha': 4.7672202249640596e-07,
  'subsample': 0.7872382459364554,
  'colsample_bytree': 0.5654477668877006,
  'n_estimators': 1000,
  'max_depth': 7,
  'min_child_weight': 4,
  'eta': 0.0977204872349886,
  'gamma': 0.22036177520409278,
  'grow_policy': 'depthwise',
  'sample_type': 'uniform',
  'normalize_type': 'forest',
  'rate_drop': 1.0763390413982438e-08,
  'skip_drop': 0.7285832530365487
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, train_null_df, test=True)

[0]	validation_0-rmse:3.71029	validation_1-rmse:3.70591
[999]	validation_0-rmse:2.00750	validation_1-rmse:2.70938
Fold: 1/3 rmse score = 2.70938
[0]	validation_0-rmse:3.70105	validation_1-rmse:3.72103
[999]	validation_0-rmse:2.01821	validation_1-rmse:2.71309
Fold: 2/3 rmse score = 2.71309
[0]	validation_0-rmse:3.71161	validation_1-rmse:3.69821
[999]	validation_0-rmse:2.01477	validation_1-rmse:2.71579
Fold: 3/3 rmse score = 2.71579


In [12]:
for idx, model in enumerate(models):
  file_name = f"./models/vwind_model_{idx}.model"
  model.save_model(file_name)

In [13]:
train_null_df["V_WIND"] = np.mean(y_tests, axis=0)

In [14]:
result = pd.concat([train_df, train_null_df], axis=0).sort_index()

In [15]:
result.head(10)

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Tanker,V_WIND,WITlt0,WTI,day,hour,minute,month,weekday,year
0,0,0,0,1,0,0,0,0,0,0,...,0,3.770000,0,-1.081668,15,4,3,10,3,2020
1,0,0,0,1,0,0,0,0,0,0,...,0,-6.720000,0,-0.188614,17,2,55,9,1,2019
2,0,0,0,1,0,0,0,0,0,0,...,0,0.000000,0,-0.305226,23,6,43,2,5,2019
3,0,0,0,0,0,0,0,1,0,0,...,0,-7.310000,0,-1.074380,18,22,6,9,4,2020
4,0,0,0,0,0,0,0,0,0,0,...,0,2.310000,0,1.209272,13,12,57,8,5,2022
5,0,0,0,1,0,0,0,0,0,0,...,0,0.194459,0,-0.840670,8,14,24,9,1,2015
6,0,0,0,0,0,0,0,1,0,0,...,0,-0.360000,0,-0.507840,25,21,34,1,0,2021
7,0,0,0,0,0,0,0,1,0,0,...,0,0.000000,0,-0.459737,18,3,48,6,1,2019
8,0,0,0,0,0,0,0,1,0,0,...,0,1.048448,0,-0.488404,27,6,23,1,4,2017
9,0,0,0,0,0,0,0,0,0,0,...,0,-0.109305,0,0.008168,3,7,8,2,5,2018


In [16]:
result.to_parquet("./data/train_v2_2v.parquet")

In [17]:
test_winds = []
for model in models:
  test_winds.append(model.predict(test_df))

In [19]:
test_df["V_WIND"] = np.mean(test_winds, axis=0)

In [20]:
assert sorted(train_df.columns) == sorted(test_df.columns)

In [21]:
test_df.to_parquet("./data/test_v2_2v.parquet")